# Анализ по объединению датасетов ATIS и restaurant8k

## Установка зависимостей

In [1]:
%pip install pandas
%pip install matplotlib
%pip install numpy


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installa

## Преподготовка

In [2]:
from dataclasses import dataclass

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import utils


In [3]:
atis_path = "../data/preprocessed/atis/"
r8k_path = "../data/preprocessed/restaurant8k/"


In [4]:
@dataclass
class DatasetSubtype:
    atis: pd.DataFrame
    r8k: pd.DataFrame


@dataclass
class Dataset:
    train: DatasetSubtype
    test: DatasetSubtype


In [5]:
dataset_train = DatasetSubtype(
    atis=pd.read_json(atis_path + "train.json"),
    r8k=pd.read_json(r8k_path + "train.json")
)

dataset_test = DatasetSubtype(
    atis=pd.read_json(atis_path + "test.json"),
    r8k=pd.read_json(r8k_path + "test.json")
)

dataset = Dataset(
    train=dataset_train, test=dataset_test
)


In [6]:
dataset.test.atis


,text,intent,entities
0,i would like to find a flight from charlotte t...,flight,"[{'start': 35, 'end': 44, 'value': 'charlotte'..."
1,on april first i need a ticket from tacoma to ...,airfare,"[{'start': 3, 'end': 8, 'value': 'april', 'ent..."
2,on april first i need a flight going from phoe...,flight,"[{'start': 3, 'end': 8, 'value': 'april', 'ent..."
3,i would like a flight traveling one way from p...,flight,"[{'start': 32, 'end': 39, 'value': 'one way', ..."
4,i would like a flight from orlando to salt lak...,flight,"[{'start': 27, 'end': 34, 'value': 'orlando', ..."
...,...,...,...
888,please find all the flights from cincinnati to...,flight,"[{'start': 33, 'end': 43, 'value': 'cincinnati..."
889,find me a flight from cincinnati to any airpor...,flight,"[{'start': 22, 'end': 32, 'value': 'cincinnati..."
890,i 'd like to fly from miami to chicago on amer...,flight,"[{'start': 22, 'end': 27, 'value': 'miami', 'e..."
891,i would like to book a round trip flight from ...,flight,"[{'start': 23, 'end': 33, 'value': 'round trip..."


Т.к. в restaurant8k нет интентов, то в результирующий датасет они не пойдут. Необходимо удалить эти поля в ATIS

In [7]:
dataset.train.atis.pop("intent")
dataset.test.atis.pop("intent")


0       flight
1      airfare
2       flight
3       flight
4       flight
        ...   
888     flight
889     flight
890     flight
891     flight
892     flight
Name: intent, Length: 893, dtype: object

In [8]:
dataset.train.atis


,text,entities
0,i want to fly from boston at 838 am and arrive...,"[{'start': 19, 'end': 25, 'value': 'boston', '..."
1,what flights are available from pittsburgh to ...,"[{'start': 32, 'end': 42, 'value': 'pittsburgh..."
2,what is the arrival time in san francisco for ...,"[{'start': 12, 'end': 24, 'value': 'arrival ti..."
3,cheapest airfare from tacoma to orlando,"[{'start': 1, 'end': 9, 'value': 'cheapest', '..."
4,round trip fares from pittsburgh to philadelph...,"[{'start': 1, 'end': 11, 'value': 'round trip'..."
...,...,...
4972,what is the airfare for flights from denver to...,"[{'start': 37, 'end': 43, 'value': 'denver', '..."
4973,do you have any flights from denver to baltimo...,"[{'start': 29, 'end': 35, 'value': 'denver', '..."
4974,which airlines fly into and out of denver,"[{'start': 35, 'end': 41, 'value': 'denver', '..."
4975,does continental fly from boston to san franci...,"[{'start': 5, 'end': 16, 'value': 'continental..."


Подготовим класс для работы с классами сущностей по аналогии с дизайном класса `Dataset`

In [9]:
@dataclass
class EntityClassesByDataset:
    atis: list
    r8k: list


@dataclass
class EntityClasses:
    train: EntityClassesByDataset
    test: EntityClassesByDataset


In [10]:
train_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.train.atis),
    r8k=utils.get_entity_classes(dataset.train.r8k)
)
test_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.test.atis),
    r8k=utils.get_entity_classes(dataset.test.r8k)
)

entity_classes = EntityClasses(
    train=train_entity_classes,
    test=test_entity_classes
)


In [11]:
entity_classes.train.r8k


['first_name', 'date', 'time', 'last_name', 'people']

## Обработка категории время

Для начала уберем некоторые части названий сущностей. Например, `depart_time` и `arrive_time`

In [12]:
time_category = []
keywords = ["time", "period_of_day"]

for keyword in keywords:
    time_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.train.atis
    )
    time_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.test.atis
    )

time_category = list(set(time_category))
time_category


['time_relative',
 'arrive_time.start_time',
 'arrive_time.time',
 'depart_time.period_mod',
 'flight_time',
 'depart_time.time_relative',
 'arrive_time.end_time',
 'depart_time.start_time',
 'period_of_day',
 'time',
 'arrive_time.period_of_day',
 'return_time.period_mod',
 'depart_time.end_time',
 'arrive_time.period_mod',
 'depart_time.time',
 'return_time.period_of_day',
 'depart_time.period_of_day',
 'arrive_time.time_relative']

In [13]:
for i in range(len(dataset.train.atis)):
    for j in range(len(dataset.train.atis["entities"][i])):
        if dataset.train.atis["entities"][i][j]["entity"] in time_category:
            if "depart_time" in dataset.train.atis["entities"][i][j]["entity"]:
                dataset.train.atis["entities"][i][j]["entity"] = dataset.train.atis["entities"][i][j]["entity"].replace(
                    "depart_time.", ""
                )
            elif "arrive_time" in dataset.train.atis["entities"][i][j]["entity"]:
                dataset.train.atis["entities"][i][j]["entity"] = dataset.train.atis["entities"][i][j]["entity"].replace(
                    "arrive_time.", ""
                )
            elif "return_time" in dataset.train.atis["entities"][i][j]["entity"]:
                dataset.train.atis["entities"][i][j]["entity"] = dataset.train.atis["entities"][i][j]["entity"].replace(
                    "return_time.", ""
                )

for i in range(len(dataset.test.atis)):
    for j in range(len(dataset.test.atis["entities"][i])):
        if dataset.test.atis["entities"][i][j]["entity"] in time_category:
            if "depart_time" in dataset.test.atis["entities"][i][j]["entity"]:
                dataset.test.atis["entities"][i][j]["entity"] = dataset.test.atis["entities"][i][j]["entity"].replace(
                    "depart_time.", ""
                )
            elif "arrive_time" in dataset.test.atis["entities"][i][j]["entity"]:
                dataset.test.atis["entities"][i][j]["entity"] = dataset.test.atis["entities"][i][j]["entity"].replace(
                    "arrive_time.", ""
                )
            elif "return_time" in dataset.test.atis["entities"][i][j]["entity"]:
                dataset.test.atis["entities"][i][j]["entity"] = dataset.test.atis["entities"][i][j]["entity"].replace(
                    "return_time.", ""
                )


In [14]:
train_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.train.atis),
    r8k=utils.get_entity_classes(dataset.train.r8k)
)
test_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.test.atis),
    r8k=utils.get_entity_classes(dataset.test.r8k)
)

entity_classes = EntityClasses(
    train=train_entity_classes,
    test=test_entity_classes
)

time_category = []
keywords = ["time", "period_of_day", "period_mod"]

for keyword in keywords:
    time_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.train.atis
    )
    time_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.test.atis
    )

time_category = list(set(time_category))
time_category


['time_relative',
 'period_of_day',
 'flight_time',
 'start_time',
 'time',
 'end_time',
 'period_mod']

In [15]:
for i in range(len(dataset.train.atis)):
    for j in range(len(dataset.train.atis["entities"][i])):
        if dataset.train.atis["entities"][i][j]["entity"] in time_category:
            if "start_time" == dataset.train.atis["entities"][i][j]["entity"]:
                dataset.train.atis["entities"][i][j]["entity"] = "time"
            elif "end_time" == dataset.train.atis["entities"][i][j]["entity"]:
                dataset.train.atis["entities"][i][j]["entity"] = "time"
            elif "period_of_day" == dataset.train.atis["entities"][i][j]["entity"]:
                dataset.train.atis["entities"][i][j]["entity"] = "time"

for i in range(len(dataset.test.atis)):
    for j in range(len(dataset.test.atis["entities"][i])):
        if dataset.test.atis["entities"][i][j]["entity"] in time_category:
            if "start_time" == dataset.test.atis["entities"][i][j]["entity"]:
                dataset.test.atis["entities"][i][j]["entity"] = "time"
            elif "end_time" == dataset.test.atis["entities"][i][j]["entity"]:
                dataset.test.atis["entities"][i][j]["entity"] = "time"
            elif "period_of_day" == dataset.test.atis["entities"][i][j]["entity"]:
                dataset.test.atis["entities"][i][j]["entity"] = "time"


In [16]:
train_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.train.atis),
    r8k=utils.get_entity_classes(dataset.train.r8k)
)
test_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.test.atis),
    r8k=utils.get_entity_classes(dataset.test.r8k)
)

entity_classes = EntityClasses(
    train=train_entity_classes,
    test=test_entity_classes
)

time_category = []
keywords = ["time", "period_of_day", "period_mod"]

for keyword in keywords:
    time_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.train.atis
    )
    time_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.test.atis
    )

time_category = list(set(time_category))
time_category


['time_relative', 'time', 'period_mod', 'flight_time']

In [17]:
date_category = []
keywords = [
    "date", "day_name", "day_number",
    "month_name", "today_relative", "year"
]

for keyword in keywords:
    date_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.train.atis
    )
    date_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.test.atis
    )

date_category = list(set(date_category))
date_category


['depart_date.year',
 'arrive_date.month_name',
 'return_date.day_name',
 'depart_date.today_relative',
 'return_date.today_relative',
 'day_number',
 'arrive_date.date_relative',
 'arrive_date.today_relative',
 'month_name',
 'depart_date.day_name',
 'return_date.month_name',
 'depart_date.month_name',
 'return_date.date_relative',
 'arrive_date.day_number',
 'depart_date.date_relative',
 'arrive_date.day_name',
 'depart_date.day_number',
 'day_name',
 'return_date.day_number',
 'today_relative']

In [18]:
for i in range(len(dataset.train.atis)):
    for j in range(len(dataset.train.atis["entities"][i])):
        if dataset.train.atis["entities"][i][j]["entity"] in date_category:
            if "depart_date" in dataset.train.atis["entities"][i][j]["entity"]:
                dataset.train.atis["entities"][i][j]["entity"] = dataset.train.atis["entities"][i][j]["entity"].replace(
                    "depart_date.", ""
                )
            elif "arrive_date" in dataset.train.atis["entities"][i][j]["entity"]:
                dataset.train.atis["entities"][i][j]["entity"] = dataset.train.atis["entities"][i][j]["entity"].replace(
                    "arrive_date.", ""
                )
            if "return_date" in dataset.train.atis["entities"][i][j]["entity"]:
                dataset.train.atis["entities"][i][j]["entity"] = dataset.train.atis["entities"][i][j]["entity"].replace(
                    "return_date.", ""
                )

for i in range(len(dataset.test.atis)):
    for j in range(len(dataset.test.atis["entities"][i])):
        if dataset.test.atis["entities"][i][j]["entity"] in date_category:
            if "depart_date" in dataset.test.atis["entities"][i][j]["entity"]:
                dataset.test.atis["entities"][i][j]["entity"] = dataset.test.atis["entities"][i][j]["entity"].replace(
                    "depart_date.", ""
                )
            elif "arrive_date" in dataset.test.atis["entities"][i][j]["entity"]:
                dataset.test.atis["entities"][i][j]["entity"] = dataset.test.atis["entities"][i][j]["entity"].replace(
                    "arrive_date.", ""
                )
            elif "return_date" in dataset.test.atis["entities"][i][j]["entity"]:
                dataset.test.atis["entities"][i][j]["entity"] = dataset.test.atis["entities"][i][j]["entity"].replace(
                    "return_date.", ""
                )


In [19]:
train_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.train.atis),
    r8k=utils.get_entity_classes(dataset.train.r8k)
)
test_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.test.atis),
    r8k=utils.get_entity_classes(dataset.test.r8k)
)

entity_classes = EntityClasses(
    train=train_entity_classes,
    test=test_entity_classes
)

date_category = []
keywords = [
    "date", "day_name", "day_number",
    "month_name", "today_relative", "year"
]

for keyword in keywords:
    date_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.train.atis
    )
    date_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.test.atis
    )

date_category = list(set(date_category))
date_category


['year',
 'month_name',
 'date_relative',
 'day_name',
 'day_number',
 'today_relative']

Нужно объединить куски даты в полноценную дату. Так, например, next tuesday из 
`date_relative` и `day_name` должны быть размечены как `date`. Отдельно стоящие `day_name` сами по себе являются датами, но если несколько сущностей идут друг за другом, то они должны объединиться в дату

In [20]:
def find_date_sequence(df, i, date_category):
    values_span = [{"start": None, "end": None}]
    positions = [{"start": None, "end": None}]
    current_pos = 0

    for item in df["entities"][i]:
        if item["entity"] in date_category:
            if values_span[len(values_span)-1]["start"] is None:
                values_span[len(values_span)-1]["start"] = item["start"]
                values_span[len(values_span)-1]["end"] = item["end"]
                
                positions[len(values_span)-1]["start"] = current_pos
                positions[len(values_span)-1]["end"] = current_pos

            else:
                values_span[len(values_span)-1]["end"] = item["end"]
                positions[len(values_span)-1]["end"] = current_pos

        else:
            if values_span[len(values_span)-1]["start"] is not None:
                positions += [{"start": None, "end": None}]
                values_span += [{"start": None, "end": None}]
        current_pos += 1

    if values_span == [{"start": None, "end": None}]:
        return None, None
    else:
        if values_span[len(values_span) - 1] == {"start": None, "end": None}:
            values_span.remove({"start": None, "end": None})
        if positions[len(positions) - 1] == {"start": None, "end": None}:
            positions.remove({"start": None, "end": None})
        return (values_span, positions)


In [21]:
values_span, pos = find_date_sequence(dataset.train.atis, 577, date_category)
print(values_span)
print(pos)
dataset.train.atis["text"][577][values_span[0]["start"]:values_span[0]["end"]]


[{'start': 52, 'end': 78}, {'start': 82, 'end': 108}]
[{'start': 2, 'end': 3}, {'start': 5, 'end': 6}]


'wednesday the twenty third'

In [22]:
for i in range(len(dataset.train.atis)):
    values_span, positions_span = find_date_sequence(
        dataset.train.atis, i, date_category
    )

    if values_span is not None:
        entities = []
        excluded_indexes = []

        for position_span in positions_span:
            excluded_indexes += list(
                np.arange(
                    position_span["start"],
                    position_span["end"] + 1, dtype=int
                )
            )

        for j in range(len(dataset.train.atis["entities"][i])):
            if j not in excluded_indexes:
                entities += [dataset.train.atis["entities"][i][j]]

        for j in range(len(values_span)):
            entity = [{
                "start": positions_span[j]["start"],
                "end": positions_span[j]["end"],
                "value": dataset.train.atis["text"][i][values_span[j]["start"]:values_span[j]["end"]],
                "entity": "date"
            }]
            entities += entity

        dataset.train.atis["entities"][i] = sorted(
            entities, key=lambda d: d['start']
        )


In [23]:
for i in range(len(dataset.test.atis)):
    values_span, positions_span = find_date_sequence(
        dataset.test.atis, i, date_category
    )

    if values_span is not None:
        entities = []
        excluded_indexes = []

        for position_span in positions_span:
            excluded_indexes += list(
                np.arange(
                    position_span["start"],
                    position_span["end"] + 1, dtype=int
                )
            )

        for j in range(len(dataset.test.atis["entities"][i])):
            if j not in excluded_indexes:
                entities += [dataset.test.atis["entities"][i][j]]

        for j in range(len(values_span)):
            entity = [{
                "start": positions_span[j]["start"],
                "end": positions_span[j]["end"],
                "value": dataset.test.atis["text"][i][values_span[j]["start"]:values_span[j]["end"]],
                "entity": "date"
            }]
            entities += entity

        dataset.test.atis["entities"][i] = sorted(
            entities, key=lambda d: d['start']
        )


In [24]:
dataset.train.atis["entities"][577]


[{'start': 2,
  'end': 3,
  'value': 'wednesday the twenty third',
  'entity': 'date'},
 {'start': 5,
  'end': 6,
  'value': 'thursday the twenty fourth',
  'entity': 'date'},
 {'start': 18,
  'end': 31,
  'value': 'san francisco',
  'entity': 'fromloc.city_name'},
 {'start': 35, 'end': 41, 'value': 'denver', 'entity': 'toloc.city_name'},
 {'start': 79, 'end': 81, 'value': 'or', 'entity': 'or'}]

In [25]:
train_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.train.atis),
    r8k=utils.get_entity_classes(dataset.train.r8k)
)
test_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.test.atis),
    r8k=utils.get_entity_classes(dataset.test.r8k)
)

entity_classes = EntityClasses(
    train=train_entity_classes,
    test=test_entity_classes
)

date_category = []
keywords = [
    "date", "day_name", "day_number",
    "month_name", "today_relative", "year"
]

for keyword in keywords:
    date_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.train.atis
    )
    date_category += utils.get_entity_classes_in_category(
        keyword, entity_classes.test.atis
    )

date_category = list(set(date_category))
date_category


['date']

In [26]:
len(entity_classes.train.atis)


46

In [27]:
@dataclass
class EntityClassesDistribution:
    train: dict
    train_size: int
    test: dict
    test_size: int
    df: pd.DataFrame


In [28]:
entity_classes_distribution_train = utils.eval_entity_classes_distribution(
    dataset.train.atis, entity_classes.train.atis
)
entities_count_train = np.array(
    list(entity_classes_distribution_train.values())
).sum()
entity_classes_distribution_train_persents = utils.eval_persentage(
    entity_classes_distribution_train, entities_count_train
)

entity_classes_distribution_test = utils.eval_entity_classes_distribution(
    dataset.test.atis, entity_classes.test.atis
)
entities_count_test = np.array(
    list(entity_classes_distribution_test.values())
).sum()
entity_classes_distribution_test_persents = utils.eval_persentage(
    entity_classes_distribution_test, entities_count_test
)

intent_classes_distribution_df = pd.DataFrame({
    "train": entity_classes_distribution_train_persents,
    "test": entity_classes_distribution_test_persents
})

entity_classes_distribution = EntityClassesDistribution(
    train=entity_classes_distribution_train_persents, train_size=entities_count_train,
    test=entity_classes_distribution_test_persents, test_size=entities_count_test,
    df=intent_classes_distribution_df
)


In [29]:
entity_classes_distribution.df

,train,test
toloc.city_name,27.268161,26.102807
fromloc.city_name,27.161424,25.665330
date,9.097759,10.572366
time,8.381993,9.223478
airline_name,4.401331,3.682100
time_relative,3.208388,3.499818
round_trip,2.184969,2.661320
cost_relative,2.159854,1.348888
flight_mod,2.065675,0.874954
stoploc.city_name,1.494318,0.729129


In [30]:
for i in range(len(dataset.train.atis)):
    for j in range(len(dataset.train.atis["entities"][i])):
        if "stoploc.state_code" == dataset.train.atis["entities"][i][j]["entity"]:
            dataset.train.atis["entities"][i][j]["entity"] = "state_code"
        elif "stoploc.airport_name" == dataset.train.atis["entities"][i][j]["entity"]:
            dataset.train.atis["entities"][i][j]["entity"] = "airport_name"
        

In [31]:
to_remove_indexes = []
for i in range(len(dataset.test.atis)):
    for j in range(len(dataset.test.atis["entities"][i])):
        if "stoploc.airport_code" == dataset.test.atis["entities"][i][j]["entity"]:
            dataset.test.atis["entities"][i][j]["entity"] = "airport_code"
        elif "flight" == dataset.test.atis["entities"][i][j]["entity"]:
            to_remove_indexes += [i]
        elif "compartment" == dataset.test.atis["entities"][i][j]["entity"]:
            to_remove_indexes += [i]
        elif "booking_class" == dataset.test.atis["entities"][i][j]["entity"]:
            to_remove_indexes += [i]


In [32]:
dataset.test.atis.drop(dataset.test.atis.index[to_remove_indexes], inplace=True)


In [33]:
dataset.test.atis.reset_index(drop=True, inplace=True) 


In [34]:
dataset.test.atis


,text,entities
0,i would like to find a flight from charlotte t...,"[{'start': 35, 'end': 44, 'value': 'charlotte'..."
1,on april first i need a ticket from tacoma to ...,"[{'start': 0, 'end': 1, 'value': 'april first'..."
2,on april first i need a flight going from phoe...,"[{'start': 0, 'end': 1, 'value': 'april first'..."
3,i would like a flight traveling one way from p...,"[{'start': 3, 'end': 4, 'value': 'april first'..."
4,i would like a flight from orlando to salt lak...,"[{'start': 2, 'end': 3, 'value': 'april first'..."
...,...,...
885,please find all the flights from cincinnati to...,"[{'start': 2, 'end': 3, 'value': 'next saturda..."
886,find me a flight from cincinnati to any airpor...,"[{'start': 22, 'end': 32, 'value': 'cincinnati..."
887,i 'd like to fly from miami to chicago on amer...,"[{'start': 22, 'end': 27, 'value': 'miami', 'e..."
888,i would like to book a round trip flight from ...,"[{'start': 23, 'end': 33, 'value': 'round trip..."


In [35]:
train_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.train.atis),
    r8k=utils.get_entity_classes(dataset.train.r8k)
)
test_entity_classes = EntityClassesByDataset(
    atis=utils.get_entity_classes(dataset.test.atis),
    r8k=utils.get_entity_classes(dataset.test.r8k)
)

entity_classes = EntityClasses(
    train=train_entity_classes,
    test=test_entity_classes
)


In [36]:
entity_classes_distribution_train = utils.eval_entity_classes_distribution(
    dataset.train.atis, entity_classes.train.atis
)
entities_count_train = np.array(
    list(entity_classes_distribution_train.values())
).sum()
entity_classes_distribution_train_persents = utils.eval_persentage(
    entity_classes_distribution_train, entities_count_train
)

entity_classes_distribution_test = utils.eval_entity_classes_distribution(
    dataset.test.atis, entity_classes.test.atis
)
entities_count_test = np.array(
    list(entity_classes_distribution_test.values())
).sum()
entity_classes_distribution_test_persents = utils.eval_persentage(
    entity_classes_distribution_test, entities_count_test
)

intent_classes_distribution_df = pd.DataFrame({
    "train": entity_classes_distribution_train_persents,
    "test": entity_classes_distribution_test_persents
})

entity_classes_distribution = EntityClassesDistribution(
    train=entity_classes_distribution_train_persents, train_size=entities_count_train,
    test=entity_classes_distribution_test_persents, test_size=entities_count_test,
    df=intent_classes_distribution_df
)


In [37]:
entity_classes_distribution.df


,train,test
toloc.city_name,27.268161,26.152158
fromloc.city_name,27.161424,25.713241
date,9.097759,10.607169
time,8.381993,9.180688
airline_name,4.401331,3.657644
time_relative,3.208388,3.511339
round_trip,2.184969,2.670080
cost_relative,2.159854,1.353328
flight_mod,2.065675,0.877835
stoploc.city_name,1.494318,0.731529


In [38]:
frames_train = [dataset.train.atis, dataset.train.r8k]
frames_test = [dataset.test.atis, dataset.test.r8k]


In [39]:
frames_train = pd.concat(frames_train)
frames_test = pd.concat(frames_test)


In [40]:
frames_train.reset_index(drop=True, inplace=True)
frames_train


,text,entities
0,i want to fly from boston at 838 am and arrive...,"[{'start': 19, 'end': 25, 'value': 'boston', '..."
1,what flights are available from pittsburgh to ...,"[{'start': 2, 'end': 2, 'value': 'thursday', '..."
2,what is the arrival time in san francisco for ...,"[{'start': 12, 'end': 24, 'value': 'arrival ti..."
3,cheapest airfare from tacoma to orlando,"[{'start': 1, 'end': 9, 'value': 'cheapest', '..."
4,round trip fares from pittsburgh to philadelph...,"[{'start': 1, 'end': 11, 'value': 'round trip'..."
...,...,...
10195,Biersack,"[{'entity': 'last_name', 'start': 0, 'end': 7,..."
10196,Four persons will be dining in our party.,"[{'entity': 'people', 'start': 0, 'end': 3, 'v..."
10197,in 12 more days,"[{'entity': 'date', 'start': 0, 'end': 14, 'va..."
10198,18,"[{'entity': 'time', 'start': 0, 'end': 1, 'val..."


In [41]:
frames_test.reset_index(drop=True, inplace=True)
frames_test


,text,entities
0,i would like to find a flight from charlotte t...,"[{'start': 35, 'end': 44, 'value': 'charlotte'..."
1,on april first i need a ticket from tacoma to ...,"[{'start': 0, 'end': 1, 'value': 'april first'..."
2,on april first i need a flight going from phoe...,"[{'start': 0, 'end': 1, 'value': 'april first'..."
3,i would like a flight traveling one way from p...,"[{'start': 3, 'end': 4, 'value': 'april first'..."
4,i would like a flight from orlando to salt lak...,"[{'start': 2, 'end': 3, 'value': 'april first'..."
...,...,...
3264,The Cazaree party of two,"[{'entity': 'last_name', 'start': 4, 'end': 10..."
3265,Use the name Anne Pegg,"[{'entity': 'first_name', 'start': 13, 'end': ..."
3266,Book under Elanor Dinkel,"[{'entity': 'first_name', 'start': 11, 'end': ..."
3267,Karla Trevizo i don't remember the phone number,"[{'entity': 'first_name', 'start': 0, 'end': 4..."


In [42]:
@dataclass
class EntityClasses:
    train: list
    test: list


In [43]:
entity_classes = EntityClasses(
    train=utils.get_entity_classes(frames_train),
    test=utils.get_entity_classes(frames_test)
)


In [44]:
entity_classes.train

['state_name',
 'flight_time',
 'round_trip',
 'fare_basis_code',
 'mod',
 'flight_number',
 'period_mod',
 'toloc.state_name',
 'fromloc.state_code',
 'flight_days',
 'time_relative',
 'toloc.city_name',
 'city_name',
 'meal_code',
 'date',
 'class_type',
 'meal',
 'time',
 'cost_relative',
 'fromloc.airport_name',
 'flight_stop',
 'restriction_code',
 'airline_name',
 'or',
 'toloc.country_name',
 'days_code',
 'aircraft_code',
 'airline_code',
 'fromloc.airport_code',
 'toloc.airport_name',
 'first_name',
 'airport_code',
 'airport_name',
 'toloc.state_code',
 'meal_description',
 'transport_type',
 'last_name',
 'state_code',
 'people',
 'fromloc.state_name',
 'fare_amount',
 'fromloc.city_name',
 'flight_mod',
 'stoploc.city_name',
 'connect',
 'toloc.airport_code',
 'economy']

In [45]:
unique_values_train = {key: [] for key in entity_classes.train}
for i in range(len(frames_train)):
    for j in range(len(frames_train["entities"][i])):
        unique_values_train[
            frames_train["entities"][i][j]["entity"]
        ] += [frames_train["entities"][i][j]["value"]]

for key in unique_values_train.keys():
    unique_values_train[key] = [list(set(unique_values_train[key]))]

unique_values_train = pd.DataFrame(unique_values_train)
unique_values_train


,state_name,flight_time,round_trip,fare_basis_code,mod,flight_number,period_mod,toloc.state_name,fromloc.state_code,flight_days,...,state_code,people,fromloc.state_name,fare_amount,fromloc.city_name,flight_mod,stoploc.city_name,connect,toloc.airport_code,economy
0,"[colorado, texas]","[times, schedule for flights, departure times,...","[round trips, round trip, one way, return]","[qx, h, qo, qx fare, q fares, q, c, yn, m, k, ...","[two, most, doesn 't, largest, same, greatest,...","[402, 71, 19, 343, 296, 210, number, 813, 279,...","[earliest, late, early]","[texas, washington, ohio, indiana, tennessee, ...",[dc],[daily],...,[dc],"[me, my sister and a colleague, me, my husband...","[texas, ohio, missouri, tennessee, florida, ca...","[1288, 1000 dollars, 100, 932 dollars, 466 dol...","[columbus, tacoma, minneapolis, ontario, westc...","[during week days, earliest possible time, dur...","[minneapolis, dallas fort worth, philadelphia,...","[connections, connect, connects, connecting, d...","[sfo, jfk, dfw, bwi, atl]","[economy, economy class]"


In [46]:
unique_values_test = {key: [] for key in entity_classes.test}
for i in range(len(frames_test)):
    for j in range(len(frames_test["entities"][i])):
        unique_values_test[
            frames_test["entities"][i][j]["entity"]
        ] += [frames_test["entities"][i][j]["value"]]

for key in unique_values_test.keys():
    unique_values_test[key] = [list(set(unique_values_test[key]))]
    

unique_values_test = pd.DataFrame(unique_values_test)
unique_values_test


,state_name,flight_time,round_trip,fare_basis_code,flight_number,mod,period_mod,toloc.state_name,fromloc.state_code,flight_days,...,state_code,people,fromloc.state_name,fare_amount,fromloc.city_name,flight_mod,stoploc.city_name,connect,toloc.airport_code,economy
0,"[arizona nevada, arizona, california, nevada]",[departure times],"[round trips, one way, return, round trip, back]","[bh, h, bn, qo, m, q, f, y]","[486, 1207, 665 673, 1500, 608, 468, 382, 19, ...","[not, closest]",[early],"[texas, ohio, washington, indiana, utah, arizo...",[dc],[daily],...,[dc],"[me and one friend, the 3 adults and 3 kids, m...","[utah, tennessee, arizona, california, minneso...","[300 dollars, 200 dollars]","[columbus, tacoma, ontario, minneapolis, salt ...","[earliest arriving, earliest, red eye, early, ...","[dallas, los angeles, columbus, san francisco,...","[connecting, direct]","[jfk, dtw, lax]",[economy]


In [47]:
entity_classes_distribution_train = utils.eval_entity_classes_distribution(
    frames_train, entity_classes.train
)
entities_count_train = np.array(
    list(entity_classes_distribution_train.values())
).sum()
entity_classes_distribution_train_persents = utils.eval_persentage(
    entity_classes_distribution_train, entities_count_train
)

entity_classes_distribution_test = utils.eval_entity_classes_distribution(
    frames_test, entity_classes.test
)
entities_count_test = np.array(
    list(entity_classes_distribution_test.values())
).sum()
entity_classes_distribution_test_persents = utils.eval_persentage(
    entity_classes_distribution_test, entities_count_test
)

intent_classes_distribution_df = pd.DataFrame({
    "train": entity_classes_distribution_train_persents,
    "test": entity_classes_distribution_test_persents
})

entity_classes_distribution = EntityClassesDistribution(
    train=entity_classes_distribution_train_persents, train_size=entities_count_train,
    test=entity_classes_distribution_test_persents, test_size=entities_count_test,
    df=intent_classes_distribution_df
)


In [48]:
entity_classes_distribution.df.head(10)

,train,test
toloc.city_name,18.432221,11.511834
fromloc.city_name,18.360071,11.318628
time,14.035311,17.774915
date,13.453866,17.581710
people,9.184280,15.826759
last_name,3.781513,6.858799
first_name,3.764536,6.649493
airline_name,2.975129,1.610047
time_relative,2.168746,1.545645
round_trip,1.476954,1.175334


In [49]:
print(
    f"Intent classes mean in dataset\n{entity_classes_distribution.df.mean()}"
)


Intent classes mean in dataset
train    2.12766
test     2.12766
dtype: float64


In [50]:
save_path = "../data/combined/"


In [51]:
frames_train.to_json(save_path + "train.json")
frames_test.to_json(save_path + "test.json")


In [52]:
unique_values_train.to_json(save_path + "unique_values_train.json")
unique_values_test.to_json(save_path + "unique_values_test.json")
